##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# linear_programming_example

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/linear_solver/linear_programming_example.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/linear_solver/samples/linear_programming_example.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [1]:
%pip install ortools

   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14
   ---------------------------------------- 0.0/18.1 MB 245.8 kB/s eta 0:01:14


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.5 which is incompatible.


In [2]:
from ortools.sat.python import cp_model

load c:\Users\alexs\anaconda3\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\alexs\anaconda3\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\alexs\anaconda3\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\alexs\anaconda3\Lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\alexs\anaconda3\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\alexs\anaconda3\Lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\alexs\anaconda3\Lib\site-packages\ortools\.libs\ortools.dll...


In [ ]:
num_products = 5
all_products = range(num_products)
num_hours = 24


forecasts = [200, 200, 50, 10, 30]
items_per_hour = [5, 2, 2, 2, 2]


# prompt: create a num_products x num_products table of cost associated with switching between two items
costs = {}
for i in all_products:
  for j in all_products:
    if i == j:
      costs[(i,j)] = 0
    else:
      costs[(i,j)] = -1

def daily_model(forecasts, feasibility=False):
  model = cp_model.CpModel()

  plan = {}

  for hour in range(num_hours):
    for product in range(num_products):
      plan[(hour, product)] = model.new_bool_var(f"shift_h{hour}_p{product}")

  # prompt: in the plan each hour should only be assigned one product
  for hour in range(num_hours):
    model.add_exactly_one(plan[(hour, product)] for product in all_products)

  # prompt: compute the sum for each product produced
  # Compute the total production for each product
  total_production = {}
  for product in all_products:
     total_production[product] = sum(plan[(hour, product)] * items_per_hour[product] for hour in range(num_hours))

  # prompt: add variable which indicates if product is switched
  is_switched = {}
  for hour in range(num_hours - 1):
    for i in all_products:
      for j in all_products:
        if i != j:
          is_switched[(hour, i, j)] = model.new_bool_var(f"is_switched_h{hour}_p{i}_to_p{j}")
          # If production switches from product i to product j at hour+1, then plan[(hour, i)] must be true and plan[(hour+1, j)] must be true.
          model.add_implication(plan[(hour, i)], is_switched[(hour, i, j)]).only_enforce_if(plan[(hour+1, j)])
          model.add_implication(plan[(hour+1, j)], is_switched[(hour, i, j)]).only_enforce_if(plan[(hour, i)])

  # substract switching cost from the total production
  for product in all_products:
    total_production[product] += sum(is_switched[(hour, i, product)] * costs[(i, product)]
                                    if i != product else 0 for hour in range(num_hours - 1)
                                    for i in all_products)

  # prompt: add constraint that total production meets the forecast
  # Add the constraint that total production for each product meets the forecast
  for product in all_products:
    model.add(total_production[product] >= forecasts[product])

  # different modes:
  # - satisfiability
  # - maximize the products produced
  # - minimize the time machines are running

  # prompt: add objective to minimize the overall number of things produced
  # Create the objective: minimize the sum of all production variables
  # model.minimize(sum(plan[(hour, product)] for hour in range(num_hours) for product in all_products))
  if not feasibility:
    model.maximize(sum(total_production[product] for product in all_products))

  # Create a solver and solve the model.
  solver = cp_model.CpSolver()
  status = solver.solve(model)

  # Print the solution.
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"Minimum total production: {solver.objective_value}")
    print("\nProduction Plan:")
    for hour in range(num_hours):
      for product in all_products:
        if solver.value(plan[(hour, product)]):
          print(f"  Hour {hour}: Produce product {product}")

    print("\nTotal production per product:")
    for product in all_products:
      print(f"Product {product}: {solver.value(total_production[product])}")

  else:
    print("No solution found.")

  print("\nStatistics")
  print(f"  Status: {solver.status_name(status)}")
  # print(f"  Conflicts: {solver.num_conflicts}")
  # print(f"  Branches: {solver.num_branches}")
  # print(f"  Wall time: {solver.wall_time} s")

  return status == cp_model.OPTIMAL or status == cp_model.FEASIBLE

In [4]:
num_days = 30

def monthly_planner(forecasts):
  model = cp_model.CpModel()

  plan = {}
  for day in range(num_days):
    for product in range(num_products):
      plan[(day, product)] = model.new_int_var(0, forecasts[product], f"shift_d{day}_p{product}")

  # Create a new constraint that makes sure that the planned amount of each item coincides with the forecast for each item
  for product in all_products:
    model.add(sum(plan[(day, product)] for day in range(num_days)) == forecasts[product])

  # heuristic constraints
  max_products_per_day = model.new_int_var(0, 200, "max_products_per_day")

  for day in range(num_days):
    model.add(sum(plan[(day, product)] for product in all_products) <= max_products_per_day)

  model.minimize(max_products_per_day)

  solver = cp_model.CpSolver()
  status = solver.solve(model)

  if not status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("We're funcked")

  daily_plan = {}
  for day in range(num_days):
    daily_plan[day] = {}
    for product in all_products:
      daily_plan[day][product] = solver.Value(plan[(day, product)])

  return daily_plan

dp = monthly_planner(forecasts)

In [5]:
all_feasible = True

print("monthly plan")

for key, value in dp.items():
  day_plan = [value[i] for i in range(num_products)]

  # print(f"Day: {key}, plan for each product: {value}")

  this_plan = daily_model(day_plan, feasibility=True)
  all_feasible = all_feasible and this_plan

# print(f"All feasible: {all_feasible}")

monthly plan
Minimum total production: 0.0

Production Plan:
  Hour 0: Produce product 1
  Hour 1: Produce product 1
  Hour 2: Produce product 1
  Hour 3: Produce product 1
  Hour 4: Produce product 4
  Hour 5: Produce product 4
  Hour 6: Produce product 1
  Hour 7: Produce product 1
  Hour 8: Produce product 1
  Hour 9: Produce product 2
  Hour 10: Produce product 2
  Hour 11: Produce product 1
  Hour 12: Produce product 1
  Hour 13: Produce product 1
  Hour 14: Produce product 1
  Hour 15: Produce product 2
  Hour 16: Produce product 2
  Hour 17: Produce product 2
  Hour 18: Produce product 2
  Hour 19: Produce product 2
  Hour 20: Produce product 2
  Hour 21: Produce product 2
  Hour 22: Produce product 2
  Hour 23: Produce product 2

Total production per product:
Product 0: 0
Product 1: 20
Product 2: 20
Product 3: 0
Product 4: 3

Statistics
  Status: OPTIMAL
Minimum total production: 0.0

Production Plan:
  Hour 0: Produce product 1
  Hour 1: Produce product 1
  Hour 2: Produce pro